In [1]:
import sys
sys.path.insert(0, "/raid/mohamed/siim-isic-melanoma-classification")
from dataset.dataset_melanoma import DatasetMelanoma as Dataset
import torchvision.transforms as transforms
from augmentation.autoaugment import ImageNetPolicy
import pandas as pd

**Prepare dataset parameters**

In [2]:
image_dir = '../data/jpeg/train/'
train_csv = pd.read_csv('../data/train_split.csv')
mean = [104.00699, 116.66877, 122.67892]
std = [0.225*255, 0.224*255, 0.229*255]
transform = transforms.Compose([transforms.Resize((256, 256)), 
                                ImageNetPolicy(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)
                               ])

**Initialize dataset**

In [3]:
dataset = Dataset(df=train_csv,
                  image_dir=image_dir,
                  transform=transform)

**Test dataset**

In [11]:
(img, data), target = dataset[10000]

In [5]:
data

tensor([1.3542, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [6]:
data.shape

torch.Size([9])

**Test EfficientNet**

In [2]:
import sys
import torch
sys.path.insert(0, "/raid/mohamed/siim-isic-melanoma-classification")
from model.efficientnet.EfficientNet import EfficientNet
inputs = torch.rand(1, 3, 224, 224)
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=2)
model.eval()
outputs = model(inputs, 1)

Loaded pretrained weights for efficientnet-b7


In [3]:
outputs

tensor([[-0.2494, -0.0815]], grad_fn=<AddmmBackward>)

In [14]:
model._fc

Linear(in_features=2560, out_features=1000, bias=True)

In [13]:
from torchsummary import summary
summary(model, (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
MemoryEfficientSwish-4         [-1, 64, 112, 112]               0
         ZeroPad2d-5         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 64, 112, 112]             576
       BatchNorm2d-7         [-1, 64, 112, 112]             128
MemoryEfficientSwish-8         [-1, 64, 112, 112]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 64, 1, 1]           1,088
         I

**TestClassifierData**

In [8]:
from model.iternet.iternet_classifier import IternetFeaturesExtractor, ClassifierData
path = '/raid/mohamed/cancer-segmentation/exp/iternet/_epoch_99.pth'
features_extractor = IternetFeaturesExtractor(path=path)

In [9]:
model = ClassifierData()

In [12]:
img_ = img.unsqueeze(0)
data_ = data.unsqueeze(0)

In [13]:
img_.shape

torch.Size([1, 3, 256, 256])

In [15]:
data_.shape

torch.Size([1, 9])

**Extract deep features**

In [19]:
features = features_extractor(img_)

**Predict using deep features and data**

In [20]:
y = model(features, data_)

-----0 torch.Size([1, 512])
-----1 torch.Size([1, 521])


In [21]:
y.shape

torch.Size([1, 2])

**Test with the classifier**

In [7]:
from model.iternet.iternet_classifier import IternetClassifier
path = '../../cancer-segmentation/exp/iternet/_epoch_99.pth'
model = IternetClassifier(num_classes=2, path=path)

**Predict based on features and meta data**

In [8]:
x = img.unsqueeze(0)
y = model(x)

In [9]:
y.shape

torch.Size([1, 2])

In [10]:
y

tensor([[ 0.0996, -0.2161]], grad_fn=<AddmmBackward>)

In [11]:
import torch
torch.nn.Softmax(dim=1)(y)

tensor([[0.5783, 0.4217]], grad_fn=<SoftmaxBackward>)